# Advanced models in `pypsa`: capacity expansion planning with sector coupling


:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa pandas numpy matplotlib plotly highspy
```
:::

## Problem description

To explore capacity expansion problem with sector-coupling options, let's model an "greenfield" energy system model with:
- electricity demand (here: Germany 2015)
- several technologies for electricity generation (wind, solar, and gas for peak load)
- hydrogen consumption (e.g., an offtaker in the industrial sector)
- hydrogen production from electrolysis
- hydrogen storage
- hydrogen fuel cell

In [19]:
import pypsa
import pandas as pd

import plotly.io as pio
import plotly.offline as py

pd.options.plotting.backend = "plotly"

## Prepare technology data

At TU Berlin, we maintain a database (https://github.com/PyPSA/technology-data) which collects assumptions and projections for energy system technologies (such as costs, efficiencies, lifetimes, etc.) for given years, which we use for our research.

Reading this data into a useable `pandas.DataFrame` requires some pre-processing (e.g. converting units, setting defaults, re-arranging dimensions):

In [20]:
YEAR = 2030
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{YEAR}.csv"
costs = pd.read_csv(url, index_col=[0, 1])

In [21]:
costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
costs.unit = costs.unit.str.replace("/kW", "/MW")

defaults = {
    "FOM": 0,
    "VOM": 0,
    "efficiency": 1,
    "fuel": 0,
    "investment": 0,
    "lifetime": 25,
    "CO2 intensity": 0,
    "discount rate": 0.07,
}
costs = costs.value.unstack().fillna(defaults)

costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["OCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]

Let's also write a small utility _function_ that calculates the **annuity** to annualise investment costs. The formula is

$$
a(r, n) = \frac{r}{1-(1+r)^{-n}}
$$
where $r$ is the discount rate and $n$ is the lifetime.

In [22]:
def annuity_factor(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

The resulting `annuity_factor` (often called capital recovery factor) represents the factor that, when multiplied by the initial capital cost, yields the equivalent annual cost accounting for the time value of money:

In [23]:
annuity_factor(0.07, 20)

0.09439292574325567

Based on this, we can calculate the short-term marginal generation costs (€/MWh)

In [24]:
costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

and the annualised investment costs (`capital_cost` in PyPSA terms, €/MW/a):

In [25]:
annuity = costs.apply(lambda x: annuity_factor(0.07, x["lifetime"]), axis=1)

In [26]:
costs["capital_cost"] = (annuity + costs["FOM"] / 100) * costs["investment"]

## Prepare load and renewable generation time-series

We are also going to need some time series for wind, solar and load.

For now, we are going to recycle the time series we used in the `pandas` tutorial.

In [27]:
url = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)

# fallback if url is not available
# ts = pd.read_csv("../resources/time-series-lecture-2.csv", index_col=0, parse_dates=True)

In [28]:
ts.head()

,load,onwind,offwind,solar,prices
2015-01-01 00:00:00,41.151,0.1566,0.7030,0.0,NaN
2015-01-01 01:00:00,40.135,0.1659,0.6875,0.0,NaN
2015-01-01 02:00:00,39.106,0.1746,0.6535,0.0,NaN
2015-01-01 03:00:00,38.765,0.1745,0.6803,0.0,NaN
2015-01-01 04:00:00,38.941,0.1826,0.7272,0.0,NaN


Let's also convert the load time series from GW to MW, the base unit of PyPSA:

In [29]:
ts.load *= 1e3

Optionally, we can downscale temporal resolution of the time series to save some computation time:

In [30]:
# here we sample only every 4th hour (reduce temporal resolution to 4-hourly)
resolution = 4
ts = ts.resample(f"{resolution}h").first()

In [31]:
ts

,load,onwind,offwind,solar,prices
2015-01-01 00:00:00,41151.0,0.1566,0.7030,0.0000,NaN
2015-01-01 04:00:00,38941.0,0.1826,0.7272,0.0000,NaN
2015-01-01 08:00:00,42963.0,0.2281,0.7828,0.0442,NaN
2015-01-01 12:00:00,47164.0,0.2384,0.8244,0.2074,NaN
2015-01-01 16:00:00,53410.0,0.3828,0.9022,0.0000,NaN
...,...,...,...,...,...
2015-12-31 04:00:00,40506.0,0.4099,0.8232,0.0000,1.64
2015-12-31 08:00:00,50223.0,0.3269,0.7597,0.0416,15.72
2015-12-31 12:00:00,50741.0,0.2319,0.7766,0.1509,26.23
2015-12-31 16:00:00,54645.0,0.1942,0.5903,0.0000,32.87


## Build our energy system to simulate
1D (time dimension; single point in space)

As always, let's initialize an empty network:

In [32]:
n = pypsa.Network()

Then, we add a single electricity bus...

In [ ]:
# No spatial component for this simple example; only one bus and no location name provided
n.add("Bus", "electricity")

...and tell the `pypsa.Network` object `n` what the snapshots of the model will be using the utility function `n.set_snapshots()`.

In [ ]:
# Add time series data to the network
n.set_snapshots(ts.index)

In [35]:
n.snapshots

DatetimeIndex(['2015-01-01 00:00:00', '2015-01-01 04:00:00',
               '2015-01-01 08:00:00', '2015-01-01 12:00:00',
               '2015-01-01 16:00:00', '2015-01-01 20:00:00',
               '2015-01-02 00:00:00', '2015-01-02 04:00:00',
               '2015-01-02 08:00:00', '2015-01-02 12:00:00',
               ...
               '2015-12-30 08:00:00', '2015-12-30 12:00:00',
               '2015-12-30 16:00:00', '2015-12-30 20:00:00',
               '2015-12-31 00:00:00', '2015-12-31 04:00:00',
               '2015-12-31 08:00:00', '2015-12-31 12:00:00',
               '2015-12-31 16:00:00', '2015-12-31 20:00:00'],
              dtype='datetime64[ns]', name='snapshot', length=2190, freq='4h')

If we resampled the time series above, we need to adjust the weighting of the snapshots (i.e. how many hours they represent). We can do that with `n.snapshot_weightings`:

In [36]:
n.snapshot_weightings.head(3)

,objective,stores,generators
snapshot,,,
2015-01-01 00:00:00,1.0,1.0,1.0
2015-01-01 04:00:00,1.0,1.0,1.0
2015-01-01 08:00:00,1.0,1.0,1.0


In [37]:
n.snapshot_weightings.loc[:, :] = resolution

In [38]:
n.snapshot_weightings.head(3)

,objective,stores,generators
snapshot,,,
2015-01-01 00:00:00,4.0,4.0,4.0
2015-01-01 04:00:00,4.0,4.0,4.0
2015-01-01 08:00:00,4.0,4.0,4.0


### Adding components for electricity part

Then, we add all the technologies we are going to include as carriers.

In [39]:
carriers = [
    "onwind",
    "offwind",
    "solar",
    "OCGT", # Open cycle gas turbine - produces electricity by burning natural gas
    "hydrogen storage underground",
]

n.add(
    "Carrier",
    carriers,
    color=["dodgerblue", "aquamarine", "gold", "indianred", "magenta"],
    co2_emissions=[costs.at[c, "CO2 intensity"] for c in carriers],
)

Next, we add the demand time series to the model.

In [40]:
n.add(
    "Load",
    "demand",
    bus="electricity",
    p_set=ts.load,
)

Let's have a check whether the data was read-in correctly.

In [41]:
n.loads_t.p_set.plot()

We are going to add one dispatchable generation technology to the model. This is an open-cycle gas turbine (OCGT) with CO$_2$ emissions of 0.2 t/MWh$_{th}$.

In [42]:
n.add(
    "Generator",
    "OCGT",
    bus="electricity",
    carrier="OCGT",
    capital_cost=costs.at["OCGT", "capital_cost"],
    marginal_cost=costs.at["OCGT", "marginal_cost"],
    efficiency=costs.at["OCGT", "efficiency"],
    p_nom_extendable=True,
)

Adding the variable renewable generators works almost identically, but we also need to supply the capacity factors to the model via the attribute `p_max_pu`.

In [25]:
for tech in ["onwind", "offwind", "solar"]:
    n.add(
        "Generator",
        tech,
        bus="electricity",
        carrier=tech,
        p_max_pu=ts[tech],
        capital_cost=costs.at[tech, "capital_cost"],
        marginal_cost=costs.at[tech, "marginal_cost"],
        efficiency=costs.at[tech, "efficiency"],
        p_nom_extendable=True,
    )

In [26]:
# Making sure the capacity factors are read-in correctly
n.generators_t.p_max_pu.loc["2015-03"].plot()

### Adding components for hydrogen part

Add a dedicated `Bus` for the hydrogen energy carrier:

In [27]:
n.add("Bus", "hydrogen", carrier="hydrogen")

Add a `Link` for the hydrogen electrolysis:

In [28]:
n.add(
    "Link",
    "electrolysis",
    bus0="electricity",
    bus1="hydrogen",
    carrier="electrolysis",
    p_nom_extendable=True,
    efficiency=0.7,
    capital_cost=50e3,  # €/MW/a
)

:::{note}
Some of the sector-coupling technologies might have multiple ouputs (e.g. CHP plants producing heat and power). PyPSA can automatically handle links have more than one input (`bus0`)
and/or output (i.e. `bus1`, `bus2`, `bus3`) with a given efficieny (`efficiency`, `efficiency2`, `efficiency3`).
:::

Add a `Link` for the fuel cell which reconverts hydrogen to electricity:

In [29]:
n.add(
    "Link",
    "fuel cell",
    bus0="hydrogen",
    bus1="electricity",
    carrier="fuel cell",
    p_nom_extendable=True,
    efficiency=0.5,
    capital_cost=120e3,  # €/MW/a
)

Add a `Store` for the hydrogen storage:

In [30]:
n.add(
    "Store",
    "hydrogen storage",
    bus="hydrogen",
    carrier="hydrogen storage",
    capital_cost=140,  # €/MWh/a
    e_nom_extendable=True,
    e_cyclic=True,  # cyclic state of charge
)

To model an industrial hydrogen offtaker, we add also a hydrogen demand to the hydrogen bus.

In the example below, we add a hydrogen demand such that it equals ~25% of the electricity demand (in MWh_H2)



In [31]:
n.add(
    "Load", "hydrogen demand", bus="hydrogen", carrier="hydrogen", p_set=19500
)  # MWh_H2/h

### We are now ready to solve the model 

In [32]:
# n.optimize.create_model()

In [33]:
n.optimize(solver_name="highs")
# 66 seconds for 1H temporal resolution (8760 snapshots)
# 7 seconds for 4H temporal resolution (2190 snapshots)

Index(['electricity', 'hydrogen'], dtype='object', name='name')
Index(['hydrogen demand'], dtype='object', name='name')
Index(['electrolysis', 'fuel cell'], dtype='object', name='name')
Index(['hydrogen storage'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 393.08it/s]
INFO:linopy.io: Writing time: 0.13s


Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-e17yrcaz has 37237 rows; 17527 cols; 71227 nonzeros
Coefficient ranges:
  Matrix  [1e-04, 4e+00]
  Cost    [4e-02, 2e+05]
  Bound   [0e+00, 0e+00]
  RHS     [2e+04, 8e+04]
Presolving model
20850 rows, 16477 cols, 53790 nonzeros  0s
Dependent equations search running on 4380 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
18660 rows, 14287 cols, 49410 nonzeros  0s
Presolve reductions: rows 18660(-18577); columns 14287(-3240); nonzeros 49410(-21817) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4380(2.44921e+08) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 17527 primals, 37237 duals
Objective: 4.75e+10
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


      14390     4.7521933007e+10 Pr: 0(0); Du: 0(7.52643e-12) 2s

Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-e17yrcaz
Model status        : Optimal
Simplex   iterations: 14390
Objective value     :  4.7521933007e+10
P-D objective error :  5.6511734793e-14
HiGHS run time      :          2.26
Writing the solution to /private/var/folders/sk/4s0w0fvj7md25wtly54j40sw0000gn/T/linopy-solve-74zq6y_g.sol


('ok', 'optimal')

### Exploring model results

The total system cost in billion Euros per year:

In [34]:
n.objective / 1e9

47.52193300731888

`n.statistics()` provides an informative overview of the model results.

See documentation: https://pypsa.readthedocs.io/en/stable/api/statistics.html

In [35]:
n.statistics()

Optimal Capacity  Installed Capacity  \
Generator OCGT                   69495.97520                 0.0   
          offwind                97743.93570                 0.0   
          solar                 147591.95957                 0.0   
Link      electrolysis           46714.11681                 0.0   
Load      -                          0.00000                 0.0   
          hydrogen                   0.00000                 0.0   
Store     hydrogen storage      938822.02685                 0.0   

                                  Supply    Withdrawal  Energy Balance  \
Generator OCGT              2.642126e+08  0.000000e+00    2.642126e+08   
          offwind           2.985578e+08  0.000000e+00    2.985578e+08   
          solar             1.601839e+08  0.000000e+00    1.601839e+08   
Link      electrolysis      0.000000e+00  2.440286e+08   -2.440286e+08   
Load      -                 0.000000e+00  4.789257e+08   -4.789257e+08   
          hydrogen          0.000000e+00  1.708200e+08   -1.708200e+08   
Store     hydrogen storage  4.916653e+07  4.916653e+07    0.000000e+00   

                           Transmission  Capacity Factor   Curtailment  \
Generator OCGT                      0.0         0.434000  3.445721e+08   
          offwind                   0.0         0.348686  1.207735e+07   
          solar                     0.0         0.123894  3.767638e+05   
Link      electrolysis              0.0         0.596332  0.000000e+00   
Load      -                         0.0              NaN  0.000000e+00   
          hydrogen                  0.0              NaN  0.000000e+00   
Store     hydrogen storage          0.0         0.158416  0.000000e+00   

                            Capital Expenditure  Operational Expenditure  \
Generator OCGT                     3.316256e+09             1.709032e+10   
          offwind                  1.706181e+10             6.329425e+06   
          solar                    7.578379e+09             1.697949e+06   
Link      electrolysis             2.335706e+09             0.000000e+00   
Load      -                        0.000000e+00             0.000000e+00   
          hydrogen                 0.000000e+00             0.000000e+00   
Store     hydrogen storage         1.314351e+08             0.000000e+00   

                                 Revenue  Market Value  
Generator OCGT              2.040657e+10     77.235416  
          offwind           1.706814e+10     57.168634  
          solar             7.580077e+09     47.321103  
Link      electrolysis     -1.362491e+10           NaN  
Load      -                -3.142989e+10           NaN  
          hydrogen         -1.609205e+10           NaN  
Store     hydrogen storage  1.314351e+08      2.673263

We can use `n.statisitcs()` to get a quick overview of optimised capacities across all components:

In [36]:
n.statistics.expanded_capacity().div(1e3).round(1)  # GW

component  carrier         
Generator  OCGT                 69.5
           offwind              97.7
           solar               147.6
Link       electrolysis         46.7
Store      hydrogen storage    938.8
dtype: float64

You can also use `n.statistics()` to promptly get an energy balance for the complete system or even any specific bus:

In [37]:
n.statistics.energy_balance(aggregate_time=False)

snapshot                                2015-01-01 00:00:00  \
component carrier          bus_carrier                        
Generator OCGT             AC                           NaN   
          offwind          AC                   68713.98680   
          onwind           AC                           NaN   
          solar            AC                           NaN   
Link      electrolysis     AC                  -27562.98680   
          fuel cell        hydrogen                     NaN   
          electrolysis     hydrogen             19294.09076   
          fuel cell        AC                           NaN   
Load      -                AC                  -41151.00000   
          hydrogen         hydrogen            -19500.00000   
Store     hydrogen storage hydrogen               205.90924   

snapshot                                2015-01-01 04:00:00  \
component carrier          bus_carrier                        
Generator OCGT             AC                           NaN   
          offwind          AC                   71079.39004   
          onwind           AC                           NaN   
          solar            AC                           NaN   
Link      electrolysis     AC                  -32138.39004   
          fuel cell        hydrogen                     NaN   
          electrolysis     hydrogen             22496.87303   
          fuel cell        AC                           NaN   
Load      -                AC                  -38941.00000   
          hydrogen         hydrogen            -19500.00000   
Store     hydrogen storage hydrogen             -2996.87303   

snapshot                                2015-01-01 08:00:00  \
component carrier          bus_carrier                        
Generator OCGT             AC                           NaN   
          offwind          AC                   76513.95286   
          onwind           AC                           NaN   
          solar            AC                    6523.56461   
Link      electrolysis     AC                  -40074.51748   
          fuel cell        hydrogen                     NaN   
          electrolysis     hydrogen             28052.16223   
          fuel cell        AC                           NaN   
Load      -                AC                  -42963.00000   
          hydrogen         hydrogen            -19500.00000   
Store     hydrogen storage hydrogen             -8552.16223   

snapshot                                2015-01-01 12:00:00  \
component carrier          bus_carrier                        
Generator OCGT             AC                           NaN   
          offwind          AC                   63267.54439   
          onwind           AC                           NaN   
          solar            AC                   30610.57241   
Link      electrolysis     AC                  -46714.11681   
          fuel cell        hydrogen                     NaN   
          electrolysis     hydrogen             32699.88177   
          fuel cell        AC                           NaN   
Load      -                AC                  -47164.00000   
          hydrogen         hydrogen            -19500.00000   
Store     hydrogen storage hydrogen            -13199.88177   

snapshot                                2015-01-01 16:00:00  \
component carrier          bus_carrier                        
Generator OCGT             AC                           NaN   
          offwind          AC                   88184.57879   
          onwind           AC                           NaN   
          solar            AC                           NaN   
Link      electrolysis     AC                  -34774.57879   
          fuel cell        hydrogen                     NaN   
          electrolysis     hydrogen             24342.20515   
          fuel cell        AC                           NaN   
Load      -                AC                  -53410.00000   
          hydrogen         

In [38]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="AC").div(1e3).groupby(
    "carrier"
).sum().T.plot()

In [39]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="hydrogen").div(
    1e3
).groupby("carrier").sum().T.plot()

Possibly, we are also interested in the total emissions:

In [40]:
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)  # t/h

In [41]:
n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6)  # Mt

127.59537376037967

## Adding emission limits

The gas power plant offers sufficient and cheap enough backup capacity to run in periods of low wind and solar generation. But what happens if this source of flexibility disappears. Let's model a 100% renewable electricity system by adding a CO$_2$ emission limit as global constraint:

In [42]:
n.add(
    "GlobalConstraint",
    "CO2Limit",
    carrier_attribute="co2_emissions",
    sense="<=",
    constant=0,
)

When we run the model now...

In [43]:
n.optimize(solver_name="highs")

Index(['electricity', 'hydrogen'], dtype='object', name='name')
Index(['hydrogen demand'], dtype='object', name='name')
Index(['electrolysis', 'fuel cell'], dtype='object', name='name')
Index(['hydrogen storage'], dtype='object', name='name')
Index(['0', '1'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 7/7 [00:00<00:00, 396.59it/s]
INFO:linopy.io: Writing time: 0.12s


Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
LP linopy-problem-um4jpv5t has 37238 rows; 17527 cols; 73417 nonzeros
Coefficient ranges:
  Matrix  [1e-04, 4e+00]
  Cost    [4e-02, 2e+05]
  Bound   [0e+00, 0e+00]
  RHS     [2e+04, 8e+04]
Presolving model
18660 rows, 14286 cols, 47220 nonzeros  0s
Dependent equations search running on 4380 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
16470 rows, 12096 cols, 42840 nonzeros  0s
Presolve reductions: rows 16470(-20768); columns 12096(-5431); nonzeros 42840(-30577) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4380(2.44921e+08) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 17527 primals, 37238 duals
Objective: 7.53e+10
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


      11327     7.5338336224e+10 Pr: 0(0); Du: 0(5.67031e-12) 2s

Performed postsolve
Solving the original LP from the solution after postsolve

Model name          : linopy-problem-um4jpv5t
Model status        : Optimal
Simplex   iterations: 11327
Objective value     :  7.5338336224e+10
P-D objective error :  6.8862527914e-15
HiGHS run time      :          1.94
Writing the solution to /private/var/folders/sk/4s0w0fvj7md25wtly54j40sw0000gn/T/linopy-solve-smj3xjwc.sol


('ok', 'optimal')

The total system cost (billion Euros per year) is now higher than before:

In [44]:
n.objective / 1e9

75.33833622431519

The optimal capacity mix now does not include any gas power plants and includes onshore wind and fuel cell technologies:

In [45]:
n.statistics.expanded_capacity().div(1e3).round(1)  # GW

component  carrier         
Generator  offwind               101.8
           onwind                111.9
           solar                 332.5
Link       electrolysis          145.0
           fuel cell             135.2
Store      hydrogen storage    38417.0
dtype: float64

Fuel cell technology steps in hours with low wind and solar generation:

In [46]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="AC").div(1e3).groupby(
    "carrier"
).sum().T.plot()

In [47]:
n.statistics.energy_balance(aggregate_time=False, bus_carrier="hydrogen").div(
    1e3
).groupby("carrier").sum().T.plot()

In [48]:
n.stores_t.e.plot()

Total emissions are now zero:

In [49]:
emissions = (
    n.generators_t.p
    / n.generators.efficiency
    * n.generators.carrier.map(n.carriers.co2_emissions)
)  # t/h

In [50]:
n.snapshot_weightings.generators @ emissions.sum(axis=1).div(1e6)  # Mt

0.0